In [ ]:
# Import necessary libraries
from flask import Flask, request, jsonify
import os
import io
import base64
from PIL import Image
import torch
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression, scale_coords
from yolov5.utils.datasets import letterbox
from yolov5.utils.torch_utils import select_device

# Initialize Flask app
app = Flask(__name__)

# Load YOLOv5 model
weights = 'yolov5s.pt'  # Path to the YOLOv5 model weights
device = 'cpu'  # Use CPU for inference
model = attempt_load(weights, map_location=device)
model.to(device).eval()

# Define classes for animal detection
classes = ['dog', 'cat', 'elephant', 'giraffe', 'horse', 'zebra']

# Define route for animal detection
@app.route('/detect', methods=['POST'])
def detect_animals():
    # Load image from POST request
    img_bytes = request.files['image'].read()
    image = Image.open(io.BytesIO(img_bytes))

    # Convert image to RGB
    image = image.convert('RGB')

    # Resize and pad image to maintain aspect ratio
    img_size = 640
    img = letterbox(image, new_shape=img_size)[0]

    # Convert image to tensor
    img = torch.from_numpy(img.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0)

    # Move image to device
    img = img.to(device)

    # Perform object detection
    with torch.no_grad():
        pred = model(img)[0]
        pred = non_max_suppression(pred, conf_thres=0.5, iou_thres=0.5)

    # Prepare response
    response = []
    for det in pred[0]:
        if det is not None:
            # Extract bounding box coordinates and class label
            x1, y1, x2, y2, conf, cls = det.tolist()
            class_label = classes[int(cls)]
            bbox = [x1, y1, x2, y2]

            # Append detection to response
            detection = {'class': class_label, 'bbox': bbox, 'conf': conf}
            response.append(detection)

    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True)
